## Setup

### Setup 1

In [108]:
import os, sys
chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"
chapter_dir = r"./" if chapter in os.listdir() else os.getcwd().split(chapter)[0]
sys.path.append(chapter_dir + f"{chapter}/exercises")

import os
os.environ["ACCELERATE_DISABLE_RICH"] = "1"
import sys
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.utils.data import DataLoader
import numpy as np
import einops
from ipywidgets import interact
import plotly.express as px
from ipywidgets import interact
from pathlib import Path
import itertools
import random
from IPython.display import display
from jaxtyping import Float, Int, Bool, Shaped, jaxtyped
from typing import List, Union, Optional, Tuple, Callable, Dict
import typeguard
from functools import partial
# from torcheval.metrics.functional import multiclass_f1_score
from sklearn.metrics import f1_score as multiclass_f1_score
import copy
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import HookedRootModule, HookPoint
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from tqdm.notebook import tqdm
from dataclasses import dataclass
from rich import print as rprint
import pandas as pd

# Make sure exercises are in the path
# exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
# section_dir = exercises_dir / "part6_othellogpt"
# section_dir = "interpretability"
# if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

from plotly_utils import imshow
from neel_plotly import scatter, line
# import part6_othellogpt.tests as tests

device = t.device("cuda" if t.cuda.is_available() else "cpu")

### Setup 2

In [109]:
MAIN = __name__ == "__main__"

cfg = HookedTransformerConfig(
    n_layers = 8,
    d_model = 512,
    d_head = 64,
    n_heads = 8,
    d_mlp = 2048,
    d_vocab = 61,
    n_ctx = 59,
    act_fn="gelu",
    normalization_type="LNPre",
    device=device,
)
model = HookedTransformer(cfg)

sd = utils.download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "synthetic_model.pth")
# champion_ship_sd = utils.download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "championship_model.pth")
model.load_state_dict(sd)

# An example input
sample_input = t.tensor([[
    20, 19, 18, 10,  2,  1, 27,  3, 41, 42, 34, 12,  4, 40, 11, 29, 43, 13, 48, 56,
    33, 39, 22, 44, 24,  5, 46,  6, 32, 36, 51, 58, 52, 60, 21, 53, 26, 31, 37,  9,
    25, 38, 23, 50, 45, 17, 47, 28, 35, 30, 54, 16, 59, 49, 57, 14, 15, 55, 7
]]).to(device)

# The argmax of the output (ie the most likely next move from each position)
sample_output = t.tensor([[
    21, 41, 40, 34, 40, 41,  3, 11, 21, 43, 40, 21, 28, 50, 33, 50, 33,  5, 33,  5,
    52, 46, 14, 46, 14, 47, 38, 57, 36, 50, 38, 15, 28, 26, 28, 59, 50, 28, 14, 28,
    28, 28, 28, 45, 28, 35, 15, 14, 30, 59, 49, 59, 15, 15, 14, 15,  8,  7,  8
]]).to(device)

assert (model(sample_input).argmax(dim=-1) == sample_output.to(device)).all()

# os.chdir(section_dir)
section_dir = Path.cwd()
sys.path.append(str(section_dir))
print(section_dir.name)

OTHELLO_ROOT = (section_dir / "othello_world").resolve()
OTHELLO_MECHINT_ROOT = (OTHELLO_ROOT / "mechanistic_interpretability").resolve()

# if not OTHELLO_ROOT.exists():
#     !git clone https://github.com/likenneth/othello_world

sys.path.append(str(OTHELLO_MECHINT_ROOT))

from mech_interp_othello_utils import (
    plot_board,
    plot_single_board,
    plot_board_log_probs,
    to_string,
    to_int,
    int_to_label,
    string_to_label,
    OthelloBoardState
)

# Load board data as ints (i.e. 0 to 60)
board_seqs_int = t.tensor(np.load(OTHELLO_MECHINT_ROOT / "board_seqs_int_small.npy"), dtype=t.long)
# Load board data as "strings" (i.e. 0 to 63 with middle squares skipped out)
board_seqs_string = t.tensor(np.load(OTHELLO_MECHINT_ROOT / "board_seqs_string_small.npy"), dtype=t.long)

assert all([middle_sq not in board_seqs_string for middle_sq in [27, 28, 35, 36]])
assert board_seqs_int.max() == 60

num_games, length_of_game = board_seqs_int.shape

# Define possible indices (excluding the four center squares)
stoi_indices = [i for i in range(64) if i not in [27, 28, 35, 36]]

# Define our rows, and the function that converts an index into a (row, column) label, e.g. `E2`
alpha = "ABCDEFGH"

def to_board_label(i):
    return f"{alpha[i//8]}{i%8}"

# Get our list of board labels
board_labels = list(map(to_board_label, stoi_indices))
full_board_labels = list(map(to_board_label, range(64)))

def plot_square_as_board(state, diverging_scale=True, **kwargs):
    """Takes a square input (8 by 8) and plot it as a board. Can do a stack of boards via facet_col=0"""
    kwargs = {
        "y": [i for i in alpha],
        "x": [str(i) for i in range(8)],
        "color_continuous_scale": "RdBu" if diverging_scale else "Blues",
        "color_continuous_midpoint": 0. if diverging_scale else None,
        "aspect": "equal",
        **kwargs
    }
    imshow(state, **kwargs)

start = 30000
num_games = 50
focus_games_int = board_seqs_int[start : start + num_games]
focus_games_string = board_seqs_string[start: start + num_games]

focus_logits, focus_cache = model.run_with_cache(focus_games_int[:, :-1].to(device))
focus_logits.shape

def one_hot(list_of_ints, num_classes=64):
    out = t.zeros((num_classes,), dtype=t.float32)
    out[list_of_ints] = 1.
    return out

focus_states = np.zeros((num_games, 60, 8, 8), dtype=np.float32)
focus_valid_moves = t.zeros((num_games, 60, 64), dtype=t.float32)

for i in (range(num_games)):
    board = OthelloBoardState()
    for j in range(60):
        board.umpire(focus_games_string[i, j].item())
        focus_states[i, j] = board.state
        focus_valid_moves[i, j] = one_hot(board.get_valid_moves())

print("focus states:", focus_states.shape)
print("focus_valid_moves", tuple(focus_valid_moves.shape))

full_linear_probe = t.load(OTHELLO_MECHINT_ROOT / "main_linear_probe.pth", map_location=device)

rows = 8
cols = 8
options = 3
assert full_linear_probe.shape == (3, cfg.d_model, rows, cols, options)

black_to_play_index = 0
white_to_play_index = 1
blank_index = 0
their_index = 1
my_index = 2

# Creating values for linear probe (converting the "black/white to play" notation into "me/them to play")
linear_probe = t.zeros(cfg.d_model, rows, cols, options, device=device)
linear_probe[..., blank_index] = 0.5 * (full_linear_probe[black_to_play_index, ..., 0] + full_linear_probe[white_to_play_index, ..., 0])
linear_probe[..., their_index] = 0.5 * (full_linear_probe[black_to_play_index, ..., 1] + full_linear_probe[white_to_play_index, ..., 2])
linear_probe[..., my_index] = 0.5 * (full_linear_probe[black_to_play_index, ..., 2] + full_linear_probe[white_to_play_index, ..., 1])

layer = 6
game_index = 0
move = 29


interpretability
focus states: (50, 60, 8, 8)
focus_valid_moves (50, 60, 64)


## Code

In [110]:
from utils import plot_game
from training_utils import get_state_stack_num_flipped
from utils import plot_probe_outputs

### Plots

In [111]:
game_index = 0
move = 4
end_move = 16
layer = 5
square = "D5"
square_tuple = (3, 5)
mine_or_their = 1

plot_game(focus_games_string, game_index=game_index, end_move = 24)
# plot_single_board(focus_games_string[game_index, :move+1], title="Original Game (black plays E0)")
# plot_single_board(focus_games_string[game_index, :move].tolist()+[to_string(to_int("C4"))], title="Corrupted Game (blank plays C0)")
'''focus_states_num_flipped = get_state_stack_num_flipped(focus_games_string)
imshow(
        focus_states_num_flipped[game_index, :end_move],
        facet_col=0,
        facet_col_wrap=8,
        facet_labels=[f"Move {i}" for i in range(0, end_move)],
        title="First 16 moves of first game",
        color_continuous_scale="Greys",
        y = [i for i in alpha],
    )'''
'''flipped_list = list(focus_states_num_flipped[game_index, :end_move, 3, 5])
first_flip = True if flipped_list[0] == 1 else False
flipped_list = [first_flip] + [flipped_list[i-1] < flipped_list[i] for i in range(1, end_move)]
print(len(flipped_list))
flipped_list = [i for i in range(0, end_move) if flipped_list[i]]
print(flipped_list)'''
plot_probe_outputs(focus_cache, linear_probe, layer, game_index, move)

### Rest

In [112]:
def cache_to_logit(cache: ActivationCache) -> float:
    resid = cache["resid_post", layer][0]
    logits= einops.einsum(resid, full_linear_probe, 'pos d_model, modes d_model rows cols options -> modes pos rows cols options')
    log_probs = logits.log_softmax(dim=-1)
    if move % 2 == 1:
        log_probs = log_probs[0] # Black to play
    else:
        log_probs = log_probs[1] # White to Play
    log_prob = log_probs[move, square_tuple[0], square_tuple[1], mine_or_their]
    return log_prob
    

In [113]:
def patching_metric(patched_cache: ActivationCache, corrupted_log_prob: float, clean_log_prob: float):
    '''
    Function of patched logits, calibrated so that it equals 0 when performance is
    same as on corrupted input, and 1 when performance is same as on clean input.

    Should be linear function of the logits for the d5 token at the final move.
    '''
    # SOLUTION
    patched_log_prob = cache_to_logit(patched_cache)
    return (patched_log_prob - corrupted_log_prob) / (clean_log_prob - corrupted_log_prob)

In [114]:
def patch_final_move_output(
    activation: Float[Tensor, "batch (head_ind) seq d_model"],
    hook: HookPoint,
    clean_cache: ActivationCache,
    head: Optional[int] = None
) -> Float[Tensor, "batch (head_ind) seq d_model"]:
    '''
    Hook function which patches activations at the final sequence position.

    Note, we only need to patch in the final sequence position, because the
    prior moves in the clean and corrupted input are identical (and this is
    an autoregressive model).
    '''
    # SOLUTION
    if head:
        activation[0, head, -1, :] = clean_cache[hook.name][0, head, -1, :]
    else:
        activation[0, -1, :] = clean_cache[hook.name][0, -1, :]
    return activation


def get_act_patch_resid_pre(
    model: HookedTransformer,
    corrupted_input: Float[Tensor, "batch pos"],
    clean_cache: ActivationCache,
    patching_metric: Callable[[ActivationCache], Float[Tensor, ""]],
    corrupted_log_prob: float,
    clean_log_prob: float
) -> Float[Tensor, "2 n_layers"]:
    '''
    Returns an array of results, corresponding to the results of patching at
    each (attn_out, mlp_out) for all layers in the model.
    '''
    # SOLUTION
    results = t.zeros(2+5, model.cfg.n_layers, device=device, dtype=t.float32)

    for layer in tqdm(range(model.cfg.n_layers)):
        for i, activation in enumerate(["attn_out", "mlp_out"]):
            hook_fn = partial(patch_final_move_output, clean_cache=clean_cache)
            model.reset_hooks()
            cache = model.add_caching_hooks()
            _ = model.run_with_hooks(
                corrupted_input,
                fwd_hooks = [(utils.get_act_name(activation, layer), hook_fn)],
            )
            cache = ActivationCache(cache, model)
            results[i, layer] = patching_metric(cache, corrupted_log_prob, clean_log_prob)
    
        for head in range(5):
            hook_fn = partial(patch_final_move_output, clean_cache=clean_cache, head=head)
            model.reset_hooks()
            cache = model.add_caching_hooks()
            _ = model.run_with_hooks(
                corrupted_input,
                fwd_hooks = [(utils.get_act_name("z", layer), hook_fn)],
            )
            cache = ActivationCache(cache, model)
            results[2+head, layer] = patching_metric(cache, corrupted_log_prob, clean_log_prob)

    return results

In [119]:
# Function takes in a tile e.g. "D3" and
# returns a list "blank" / "mine" / "their" with length 60
# For every move it sais whether the tile is blank, mine or their
def get_tile_state(square, game_index):
    tile_index = to_string(to_int(square))
    white_or_black = focus_states[game_index, :, tile_index // 8, tile_index % 8].copy() # 0 is blank, -1 is white, 1 is black
    # change dtype to int
    white_or_black = white_or_black.astype(int)
    print(white_or_black)
    assert len(white_or_black.shape) == 1
    assert white_or_black.shape[0] == 60
    white_or_black[0::2] *= -1 # 0 blank, 1 mine, -1 theirs
    white_or_black[white_or_black == 1] = 2
    white_or_black[white_or_black == -1] = 1
    print(white_or_black)
    assert set(white_or_black) == set([0, 1, 2])
    return list(white_or_black) # 0 blank, 1 their, 2 mine

In [123]:
game_index = 0
move = 4
end_move = 16
layer = 5
square = "D5"
tile_index = to_string(to_int(square))
square_tuple = (tile_index // 8, tile_index % 8)
print(square_tuple)
# Not Correct :(
mine_or_their_list = get_tile_state(square, game_index)
print(mine_or_their_list)
mine_or_their = mine_or_their_list[move]
print(mine_or_their)
print(mine_or_their_list[15])
# I think 2 is actually correct. Something is really weird
mine_or_their = 1

clean_input = focus_games_int[game_index, :move+1].clone()
corrupted_input = focus_games_int[game_index, :move+1].clone()
corrupted_input[3] = to_int("F3")
print("Clean:     ", ", ".join(int_to_label(clean_input)))
print("Corrupted: ", ", ".join(int_to_label(corrupted_input)))

_, clean_cache = model.run_with_cache(clean_input)
_, corrupted_cache = model.run_with_cache(corrupted_input)

clean_log_prob = cache_to_logit(clean_cache)
corrupted_log_prob = cache_to_logit(corrupted_cache)

print(f"Clean log prob of {square} at move {move}: {clean_log_prob.item()}")
print(f"Corrupted log prob of {square} at move {move}: {corrupted_log_prob.item()}")

patching_results = get_act_patch_resid_pre(model, corrupted_input, clean_cache, patching_metric, corrupted_log_prob, clean_log_prob)

line(patching_results, title="Layer Output Patching Effect on D5 Log Prob", line_labels=["attn", "mlp"] + [f"head_{head}" for head in range(5)], width=750)

(3, 5)
[ 0  0  0 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[0 0 0 1 2 1 2 1 2 1 1 1 2 1 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1
 2 1 2 1 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1]
[0, 0, 0, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1]
2
2
Clean:      C3, C2, C1, D5, F5
Corrupted:  C3, C2, C1, F3, F5
Clean log prob of D5 at move 4: -3.123234637314454e-05
Corrupted log prob of D5 at move 4: -21.72783088684082


  0%|          | 0/8 [00:00<?, ?it/s]

### Old

In [ ]:
model.reset_hooks()
hook_fn = partial(patch_final_move_output, clean_cache=clean_cache)
cache = model.add_caching_hooks()
_ = model.run_with_hooks(
    corrupted_input,
    fwd_hooks = [
        (utils.get_act_name("attn_out", 0), hook_fn),
        (utils.get_act_name("attn_out", 4), hook_fn)]
)
cache = ActivationCache(cache, model)
print(cache.keys())
metric = patching_metric(cache)
print(metric)
# print(cache)'''

dict_keys(['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'blocks.2.hook_resid_pre', 'block

TypeError: patching_metric() missing 2 required positional arguments: 'corrupted_log_prob' and 'clean_log_prob'

In [ ]:
def patch_final_move_output(
    activation: Float[Tensor, "batch seq d_model"],
    hook: HookPoint,
    clean_cache: ActivationCache,
) -> Float[Tensor, "batch seq d_model"]:
    '''
    Hook function which patches activations at the final sequence position.

    Note, we only need to patch in the final sequence position, because the
    prior moves in the clean and corrupted input are identical (and this is
    an autoregressive model).
    '''
    # SOLUTION
    activation[0, -1, :] = clean_cache[hook.name][0, -1, :]
    return activation


def get_act_patch_resid_pre(
    model: HookedTransformer,
    corrupted_input: Float[Tensor, "batch pos"],
    clean_cache: ActivationCache,
    patching_metric: Callable[[ActivationCache], Float[Tensor, ""]],
    corrupted_log_prob: float,
    clean_log_prob: float
) -> Float[Tensor, "2 n_layers"]:
    '''
    Returns an array of results, corresponding to the results of patching at
    each (attn_out, mlp_out) for all layers in the model.
    '''
    # SOLUTION
    model.reset_hooks()
    cache = model.add_caching_hooks()
    results = t.zeros(2, model.cfg.n_layers, device=device, dtype=t.float32)
    hook_fn = partial(patch_final_move_output, clean_cache=clean_cache)

    for i, activation in enumerate(["attn_out", "mlp_out"]):
        for layer in tqdm(range(model.cfg.n_layers)):
            _ = model.run_with_hooks(
                corrupted_input,
                fwd_hooks = [(utils.get_act_name(activation, layer), hook_fn)],
            )
            cache = ActivationCache(cache, model)
            results[i, layer] = patching_metric(cache, corrupted_log_prob, clean_log_prob)

    return results